In [1]:
import pandas as pd
import numpy as np
import os, sys
import cv2
import matplotlib.pyplot as plt
import copy
import tensorflow as tf
from io import StringIO # Python3 use: from io import StringIO

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

DATA = os.path.join(module_path, 'Data', 'Augmented Data')
RESULTS = os.path.join(module_path, 'Results', 'Thesis')
FIGURES = os.path.join(module_path, 'Figures', 'Thesis')
from Scripts import Data_Loader_Functions as dL
from Scripts import Model_Architectures as mA
from Scripts import Results_Evaluation as rE
from Scripts import Print_Functions as pF

In [2]:
GROUP_2_PATH = os.path.join(DATA, "group_2")
CENTRAL_MODELS = os.path.join(module_path, 'Models', 'Pain', 'Centralized')
FEDERATED_MODELS = os.path.join(module_path, 'Models', 'Pain', 'Federated')

In [3]:
df = dL.create_pain_df(GROUP_2_PATH)
# df = df[(df['Trans_1'] == 'original') & (df['Trans_2'] == 'straight')]

In [4]:
tf.test.is_gpu_available()

W0206 03:00:42.084549 140547203680000 deprecation.py:323] From <ipython-input-4-17bb7203622b>:1: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
concat_df = pd.DataFrame()
        
for session in df['Session'].unique():
    print("Session {}".format(session))
    df_session = df[df['Session'] == session]
    sess_data, sess_labels, sess_people, sess_all_labels = dL.load_and_prepare_pain_data(
            df_session['img_path'].values,
            person=0,
            pain=4,
            model_type='CNN')

    for model_path in os.listdir(CENTRAL_MODELS):
        if "0.00" in model_path:
            seed = model_path.split('training_')[1].split('_shard')[0]
            model = tf.keras.models.load_model(os.path.join(CENTRAL_MODELS, model_path))
            results_df = pd.DataFrame(sess_all_labels, columns=['Subject ID', 'Session', 'a', 'Frame', 'True Label', 'd', 'e']).drop(columns=['a', 'd', 'e'])
            results_df['BCDL'] = model.predict(sess_data)
            results_df['Seed'] = int(seed)
            concat_df = pd.concat((concat_df, results_df))
            del model

Session 0
0 images processed
1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed
11000 images processed
12000 images processed
13000 images processed
14000 images processed
Normalization
Session 1
0 images processed
1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed
11000 images processed
Normalization
Session 2
0 images processed
1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed
Normalization
Session 3
0 images processed
1000 images processed
2000 images processed
3000 images processe

In [6]:
# concat_df.to_csv('BCDL.csv')
concat_df.to_csv('BCDL_all.csv')

# CDL

In [8]:
concat_df = pd.DataFrame()
for session in df['Session'].unique():
    print("Session {}".format(session))


    df_session = df[df['Session'] == session]
    sess_data, sess_labels, sess_people, sess_all_labels = dL.load_and_prepare_pain_data(
            df_session['img_path'].values,
            person=0,
            pain=4,
            model_type='CNN')
        
    for model_path in os.listdir(CENTRAL_MODELS):
        if '.h5' in model_path and session == int(float(model_path.split('shard-')[1].split('.h5')[0])):
            print("Model: {}".format(model_path))

            seed = model_path.split('training_')[1].split('_shard')[0]
            model = tf.keras.models.load_model(os.path.join(CENTRAL_MODELS, model_path))
            results_df = pd.DataFrame(sess_all_labels, columns=['Subject ID', 'Session', 'a', 'Frame', 'True Label', 'd', 'e']).drop(columns=['a', 'd', 'e'])
            results_df['CDL'] = model.predict(sess_data)
            results_df['Seed'] = int(seed)
            concat_df = pd.concat((concat_df, results_df))
            model = None
            del model

Session 0
0 images processed
1000 images processed
2000 images processed
3000 images processed
Normalization
Model: 2020-01-26-144618_PAIN_2-sessions-Centralized-pre-training_130_shard-0.00.h5
Model: 2020-01-26-184558_PAIN_2-sessions-Centralized-pre-training_132_shard-0.00.h5
Model: 2020-01-26-011953_PAIN_2-sessions-Centralized-pre-training_124_shard-0.00.h5
Model: 2020-01-26-055919_PAIN_2-sessions-Centralized-pre-training_126_shard-0.00.h5
Model: 2020-01-26-075707_PAIN_2-sessions-Centralized-pre-training_127_shard-0.00.h5
Model: 2020-01-26-101947_PAIN_2-sessions-Centralized-pre-training_128_shard-0.00.h5
Model: 2020-01-25-230619_PAIN_2-sessions-Centralized-pre-training_123_shard-0.00.h5
Model: 2020-01-26-123625_PAIN_2-sessions-Centralized-pre-training_129_shard-0.00.h5
Model: 2020-01-26-034045_PAIN_2-sessions-Centralized-pre-training_125_shard-0.00.h5
Model: 2020-01-26-163757_PAIN_2-sessions-Centralized-pre-training_131_shard-0.00.h5
Session 1
0 images processed
1000 images processed


In [9]:
# concat_df.to_csv('CDL.csv')
concat_df.to_csv('CDL_all.csv')

# LDL

In [10]:
concat_df = pd.DataFrame()
for session in df['Session'].unique():
    print("Session {}".format(session))


    df_session = df[df['Session'] == session]
    sess_data, sess_labels, sess_people, sess_all_labels = dL.load_and_prepare_pain_data(
            df_session['img_path'].values,
            person=0,
            pain=4,
            model_type='CNN')
        
    for model_path in os.listdir(FEDERATED_MODELS):
        if '.h5' in model_path and session == int(float(model_path.split('shard-')[1].split('.h5')[0])):
            if 'local-models' in model_path:
                print("Model: {}".format(model_path))

                seed = model_path.split('local-models_')[1].split('_shard')[0]
                model = tf.keras.models.load_model(os.path.join(FEDERATED_MODELS, model_path))
                results_df = pd.DataFrame(sess_all_labels, columns=['Subject ID', 'Session', 'a', 'Frame', 'True Label', 'd', 'e']).drop(columns=['a', 'd', 'e'])
                results_df['LDL'] = model.predict(sess_data)
                results_df['Seed'] = int(seed)
                concat_df = pd.concat((concat_df, results_df))
                model = None
                del model

Session 0
0 images processed
1000 images processed
2000 images processed
3000 images processed
Normalization
Session 1
0 images processed
1000 images processed
2000 images processed
Normalization
Model: 2020-01-26-025742_PAIN_10-sessions-Federated-central-pre-training-local-models_124_shard-1.h5
Model: 2020-01-27-021932_PAIN_10-sessions-Federated-central-pre-training-local-models_132_shard-1.h5
Model: 2020-01-26-093655_PAIN_10-sessions-Federated-central-pre-training-local-models_127_shard-1.h5
Model: 2020-01-26-003838_PAIN_10-sessions-Federated-central-pre-training-local-models_123_shard-1.h5
Model: 2020-01-26-071850_PAIN_10-sessions-Federated-central-pre-training-local-models_126_shard-1.h5
Model: 2020-01-26-115826_PAIN_10-sessions-Federated-central-pre-training-local-models_128_shard-1.h5
Model: 2020-01-26-180639_PAIN_10-sessions-Federated-central-pre-training-local-models_131_shard-1.h5
Model: 2020-01-26-140333_PAIN_10-sessions-Federated-central-pre-training-local-models_129_shard-1

In [11]:
concat_df.to_csv('LDL_all.csv')

# FDL

In [5]:
concat_df = pd.DataFrame()
for session in df['Session'].unique():
    print("Session {}".format(session))


    df_session = df[df['Session'] == session]
    sess_data, sess_labels, sess_people, sess_all_labels = dL.load_and_prepare_pain_data(
            df_session['img_path'].values,
            person=0,
            pain=4,
            model_type='CNN')
        
    for model_path in os.listdir(FEDERATED_MODELS):
        if '.h5' in model_path and session == int(float(model_path.split('shard-')[1].split('.h5')[0])):
            if 'training_' in model_path:
                print("Model: {}".format(model_path))

                seed = model_path.split('training_')[1].split('_shard')[0]
                model = tf.keras.models.load_model(os.path.join(FEDERATED_MODELS, model_path))
                results_df = pd.DataFrame(sess_all_labels, columns=['Subject ID', 'Session', 'a', 'Frame', 'True Label', 'd', 'e']).drop(columns=['a', 'd', 'e'])
                results_df['FDL'] = model.predict(sess_data)
                results_df['Seed'] = int(seed)
                concat_df = pd.concat((concat_df, results_df))
                model = None
                del model

Session 0
0 images processed
1000 images processed
2000 images processed
3000 images processed
Normalization
Session 1
0 images processed
1000 images processed
2000 images processed
Normalization
Model: 2020-01-26-060945_PAIN_4-sessions-Federated-central-pre-training_126_shard-1.h5
Model: 2020-01-26-035149_PAIN_4-sessions-Federated-central-pre-training_125_shard-1.h5
Model: 2020-01-26-080707_PAIN_4-sessions-Federated-central-pre-training_127_shard-1.h5
Model: 2020-01-25-231737_PAIN_4-sessions-Federated-central-pre-training_123_shard-1.h5
Model: 2020-01-26-103051_PAIN_4-sessions-Federated-central-pre-training_128_shard-1.h5
Model: 2020-01-26-124732_PAIN_4-sessions-Federated-central-pre-training_129_shard-1.h5
Model: 2020-01-26-164756_PAIN_4-sessions-Federated-central-pre-training_131_shard-1.h5
Model: 2020-01-27-005514_PAIN_4-sessions-Federated-central-pre-training_132_shard-1.h5
Model: 2020-01-26-013032_PAIN_4-sessions-Federated-central-pre-training_124_shard-1.h5
Model: 2020-01-26-145

In [6]:
concat_df.to_csv('FDL_all.csv')

# PFDL

In [14]:
concat_df = pd.DataFrame()
for session in df['Session'].unique():
    print("Session {}".format(session))


    df_session = df[df['Session'] == session]
    sess_data, sess_labels, sess_people, sess_all_labels = dL.load_and_prepare_pain_data(
            df_session['img_path'].values,
            person=0,
            pain=4,
            model_type='CNN')
        
    for model_path in os.listdir(FEDERATED_MODELS):
        if '.h5' in model_path and session == int(float(model_path.split('shard-')[1].split('.h5')[0])):
            if 'personalization' in model_path:
                print("Model: {}".format(model_path))

                seed = model_path.split('personalization_')[1].split('_shard')[0]
                model = tf.keras.models.load_model(os.path.join(FEDERATED_MODELS, model_path))
                results_df = pd.DataFrame(sess_all_labels, columns=['Subject ID', 'Session', 'a', 'Frame', 'True Label', 'd', 'e']).drop(columns=['a', 'd', 'e'])
                results_df['PFDL'] = model.predict(sess_data)
                results_df['Seed'] = int(seed)
                concat_df = pd.concat((concat_df, results_df))
                model = None
                del model

Session 0
0 images processed
1000 images processed
2000 images processed
3000 images processed
Normalization
Session 1
0 images processed
1000 images processed
2000 images processed
Normalization
Model: 2020-01-26-131602_PAIN_7-sessions-Federated-central-pre-training-personalization_129_shard-1.h5
Model: 2020-01-26-111150_PAIN_7-sessions-Federated-central-pre-training-personalization_128_shard-1.h5
Model: 2020-01-26-084619_PAIN_7-sessions-Federated-central-pre-training-personalization_127_shard-1.h5
Model: 2020-01-27-013103_PAIN_7-sessions-Federated-central-pre-training-personalization_132_shard-1.h5
Model: 2020-01-26-043011_PAIN_7-sessions-Federated-central-pre-training-personalization_125_shard-1.h5
Model: 2020-01-25-235020_PAIN_7-sessions-Federated-central-pre-training-personalization_123_shard-1.h5
Model: 2020-01-26-020856_PAIN_7-sessions-Federated-central-pre-training-personalization_124_shard-1.h5
Model: 2020-01-26-153352_PAIN_7-sessions-Federated-central-pre-training-personaliza

In [15]:
concat_df.to_csv('PFDL_all.csv')